<a href="https://colab.research.google.com/github/ViacheslavCRBP/pythonProject/blob/master/HW_NLP_05_ViacheslavS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
!pip install pyconll

In [33]:
!mkdir datasets
# SynTagRus is now split into 3 parts. Download them all
!wget -O ./datasets/ru_syntagrus-ud-train-a.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-a.conllu
!wget -O ./datasets/ru_syntagrus-ud-train-b.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-b.conllu
!wget -O ./datasets/ru_syntagrus-ud-train-c.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-c.conllu
# Concatenate files together to get the full training set
!cat ./datasets/ru_syntagrus-ud-train-a.conllu ./datasets/ru_syntagrus-ud-train-b.conllu ./datasets/ru_syntagrus-ud-train-c.conllu > ./datasets/ru_syntagrus-ud-train.conllu
!wget -O ./datasets/ru_syntagrus-ud-dev.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu

mkdir: cannot create directory ‘datasets’: File exists
--2023-12-15 15:51:36--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-a.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 40736599 (39M) [text/plain]
Saving to: ‘./datasets/ru_syntagrus-ud-train-a.conllu’

./datasets/ru_synta 100%[===================>]  38.85M   121MB/s    in 0.3s    

2023-12-15 15:51:37 (121 MB/s) - ‘./datasets/ru_syntagrus-ud-train-a.conllu’ saved [40736599/40736599]

--2023-12-15 15:51:37--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-b.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133

In [34]:
import pyconll
import nltk
from nltk.tag import DefaultTagger
from nltk.tag import UnigramTagger
from nltk.tag import BigramTagger, TrigramTagger

from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

from sklearn.preprocessing import LabelEncoder

In [35]:
full_train = pyconll.load_from_file('datasets/ru_syntagrus-ud-train.conllu')
full_test = pyconll.load_from_file('datasets/ru_syntagrus-ud-dev.conllu')

In [36]:
fdata_train = []
for sent in full_train[:]:
    fdata_train.append([(token.form, token.upos) for token in sent])

fdata_test = []
for sent in full_test[:]:
    fdata_test.append([(token.form, token.upos) for token in sent])

fdata_sent_test = []
for sent in full_test[:]:
    fdata_sent_test.append([token.form for token in sent])

In [37]:
all_train_texts = [' '.join(token.form if token.form is not None  else '' for token in sent) for sent in full_train]
all_test_texts = [' '.join(token.form if token.form is not None  else '' for token in sent) for sent in full_test]

all_train_labels = [' '.join(token.form if token.form is not None  else '' for token in sent) for sent in full_train]
all_test_labels = [' '.join(token.form if token.form is not None  else '' for token in sent) for sent in full_test]
print('\n'.join(all_train_texts[10:20]))
print('\n'.join(all_train_labels[10:20]))

Семен Еремеевич очень не любил , когда за этот стол кто-нибудь садился , и если видел отодвинутый стул , то всегда собственноручно подвигал его на место , так чтобы спинки образовывали ровную прямую линию .
В тот день , с которого начинается наш рассказ , жизнь в приемной протекала обычно .
Секретарша ставила сургучные печати на пакет , посетители ожидали своей очереди , радио играло сентиментальный вальс .
Ровно в десять часов раздался короткий звонок .
Быстрая секретарша на минуту скрылась в кабинете и проговорила , вернувшись : - Товарищ Ефимова , пожалуйста .
Молодая женщина в кокетливой шляпке прошла в кабинет .
Она , видимо , волновалась , потому что забыла закрыть за собой обе обитые черной клеенкой двери .
Семен Еремеевич медленно писал , склонив по привычке голову набок .
- Садитесь , пожалуйста , - сказал он , не поднимая глаз .
Ефимова села на краешек холодного кресла и посмотрела сквозь бронзовый забор чернильного прибора на Семена Еремеевича .
Семен Еремеевич очень не люби

In [38]:
test_sent = all_test_texts[15]
test_data = fdata_test

In [39]:
unigram_tagger = UnigramTagger(fdata_train)
display(unigram_tagger.tag(test_sent.lower().split()), unigram_tagger.evaluate(test_data))

<ipython-input-39-7b3ae53b9194>:2: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  display(unigram_tagger.tag(test_sent.lower().split()), unigram_tagger.evaluate(test_data))


[('различные', 'ADJ'),
 ('определения', 'NOUN'),
 ('алгоритма', None),
 ('в', 'ADP'),
 ('явной', 'ADJ'),
 ('или', 'CCONJ'),
 ('неявной', 'ADJ'),
 ('форме', 'NOUN'),
 ('содержат', 'VERB'),
 ('следующий', 'ADJ'),
 ('ряд', 'NOUN'),
 ('общих', 'ADJ'),
 ('требований', 'NOUN'),
 (':', 'PUNCT')]

0.8782863467673677

In [40]:
bigram_tagger = BigramTagger(fdata_train)
display(bigram_tagger.tag(test_sent.lower().split()), bigram_tagger.evaluate(test_data))

<ipython-input-40-2a8b5d872da2>:2: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  display(bigram_tagger.tag(test_sent.lower().split()), bigram_tagger.evaluate(test_data))


[('различные', None),
 ('определения', None),
 ('алгоритма', None),
 ('в', 'ADP'),
 ('явной', None),
 ('или', 'CCONJ'),
 ('неявной', 'ADJ'),
 ('форме', 'NOUN'),
 ('содержат', 'VERB'),
 ('следующий', 'ADJ'),
 ('ряд', 'NOUN'),
 ('общих', 'ADJ'),
 ('требований', 'NOUN'),
 (':', 'PUNCT')]

0.7101308678950452

In [41]:
bigram_tagger = BigramTagger(fdata_train, backoff=unigram_tagger)
display(bigram_tagger.tag(test_sent.lower().split()), bigram_tagger.evaluate(test_data))

<ipython-input-41-ad44b3ca648e>:2: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  display(bigram_tagger.tag(test_sent.lower().split()), bigram_tagger.evaluate(test_data))


[('различные', 'ADJ'),
 ('определения', 'NOUN'),
 ('алгоритма', None),
 ('в', 'ADP'),
 ('явной', 'ADJ'),
 ('или', 'CCONJ'),
 ('неявной', 'ADJ'),
 ('форме', 'NOUN'),
 ('содержат', 'VERB'),
 ('следующий', 'ADJ'),
 ('ряд', 'NOUN'),
 ('общих', 'ADJ'),
 ('требований', 'NOUN'),
 (':', 'PUNCT')]

0.8839768214076438

In [42]:
trigram_tagger = TrigramTagger(fdata_train)
display(trigram_tagger.tag(test_sent.lower().split()), trigram_tagger.evaluate(test_data))

<ipython-input-42-cfbff5572b15>:2: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  display(trigram_tagger.tag(test_sent.lower().split()), trigram_tagger.evaluate(test_data))


[('различные', None),
 ('определения', None),
 ('алгоритма', None),
 ('в', 'ADP'),
 ('явной', None),
 ('или', None),
 ('неявной', None),
 ('форме', None),
 ('содержат', None),
 ('следующий', None),
 ('ряд', None),
 ('общих', None),
 ('требований', None),
 (':', None)]

0.4067191874470994

In [43]:
trigram_tagger = TrigramTagger(fdata_train, backoff=unigram_tagger)
display(trigram_tagger.tag(test_sent.lower().split()), trigram_tagger.evaluate(test_data))

<ipython-input-43-448a67824170>:2: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  display(trigram_tagger.tag(test_sent.lower().split()), trigram_tagger.evaluate(test_data))


[('различные', 'ADJ'),
 ('определения', 'NOUN'),
 ('алгоритма', None),
 ('в', 'ADP'),
 ('явной', 'ADJ'),
 ('или', 'CCONJ'),
 ('неявной', 'ADJ'),
 ('форме', 'NOUN'),
 ('содержат', 'VERB'),
 ('следующий', 'ADJ'),
 ('ряд', 'NOUN'),
 ('общих', 'ADJ'),
 ('требований', 'NOUN'),
 (':', 'PUNCT')]

0.8827137183410378

In [44]:
trigram_tagger = TrigramTagger(fdata_train, backoff=bigram_tagger)
display(trigram_tagger.tag(test_sent.lower().split()), trigram_tagger.evaluate(test_data))

<ipython-input-44-3ddf8c88813c>:2: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  display(trigram_tagger.tag(test_sent.lower().split()), trigram_tagger.evaluate(test_data))


[('различные', 'ADJ'),
 ('определения', 'NOUN'),
 ('алгоритма', None),
 ('в', 'ADP'),
 ('явной', 'ADJ'),
 ('или', 'CCONJ'),
 ('неявной', 'ADJ'),
 ('форме', 'NOUN'),
 ('содержат', 'VERB'),
 ('следующий', 'ADJ'),
 ('ряд', 'NOUN'),
 ('общих', 'ADJ'),
 ('требований', 'NOUN'),
 (':', 'PUNCT')]

0.8830522820496126

In [45]:
train_tok = []
train_label = []
for sent in fdata_train[:]:
    for tok in sent:
        if (tok is None or tok[0] is None):
          continue
        train_tok.append(tok[0])
        train_label.append('NO_TAG' if tok[1] is None else tok[1])

test_tok = []
test_label = []
for sent in fdata_test[:]:
    for tok in sent:
        if (tok is None or tok[0] is None):
          continue
        test_tok.append(tok[0])
        test_label.append('NO_TAG' if tok[1] is None else tok[1])

In [46]:
le = LabelEncoder()
train_enc_labels = le.fit_transform(train_label)

In [47]:
test_enc_labels = le.transform(test_label)

In [48]:
hvectorizer = HashingVectorizer(ngram_range=(1, 3), analyzer='word')
X_train = hvectorizer.fit_transform(train_tok)
X_test = hvectorizer.transform(test_tok)

In [49]:
lr = LogisticRegression(random_state=0, max_iter=100)
lr.fit(X_train, train_enc_labels)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=0)

In [50]:
pred = lr.predict(X_test)
accuracy_score(test_enc_labels, pred)

0.6554071217815527

In [51]:
hvectorizer = HashingVectorizer(ngram_range=(1, 4), analyzer='char_wb', n_features=500)
X_train = hvectorizer.fit_transform(train_tok)
X_test = hvectorizer.transform(test_tok)

In [52]:
lr = LogisticRegression(random_state=0, max_iter=100)
lr.fit(X_train, train_enc_labels)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=0)

In [53]:
pred = lr.predict(X_test)

In [54]:
accuracy_score(test_enc_labels, pred)

0.8652436998873178

In [55]:
tvectorizer = TfidfVectorizer(ngram_range=(1, 2), analyzer='char_wb')
X_train = tvectorizer.fit_transform(train_tok)
X_test = tvectorizer.transform(test_tok)

In [56]:
lr = LogisticRegression(random_state=0, max_iter=100)
lr.fit(X_train, train_enc_labels)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=0)

In [57]:
pred = lr.predict(X_test)
accuracy_score(test_enc_labels, pred)

0.8824521751590905

In [62]:
tvectorizer = TfidfVectorizer(ngram_range=(1, 3), analyzer='word')
X_train = tvectorizer.fit_transform(train_tok)
X_test = tvectorizer.transform(test_tok)

In [63]:
lr = LogisticRegression(random_state=0, max_iter=100)
lr.fit(X_train, train_enc_labels)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=0)

In [64]:
pred = lr.predict(X_test)
accuracy_score(test_enc_labels, pred)

0.6587745637631979

Выводы:

1) UnigramTagger - 0.8783  
2) BigramTagger - 0.7101  
3) BigramTagger + UnigramTagger - 0.884  
4) TrigramTagger - 0.8831  
5) Свой (HashingVectorizer(word, 1-3, max feat) + LR 100 итераций) - 0.6554  
6) Свой (HashingVectorizer(char, 1-4, 500 feat) + LR 100 итераций) - 0.8652   
7) Свой (TfidfVectorizer(char, 1-2, max feat) + LR 100 итераций) - 0.8825  
8) Свой (TfidfVectorizer(word, 1-3, max feat) + LR 100 итераций) - 0.6587  

Результат выше у сети, которая обучалась на N-граммах.

Готовые решения NER из spacy и nltk показывают хорошие результаты в задачах извлечения именованных сущностей.

Вывод: наилучшие результаты даёт комбинация Unigram+Bigram и линейная регрессия с любым векторизатором без ограничения на кол-во фичей. Линейная регрессия работает при этом заметно медленнее, но даёт +5% точности. Попытки векторизировать данные не по символам, а по словам приводят к сильному ухудшению точности. Можно предположить, что это связано с тем, что при использовании символьных n-грам, модель может научиться понимать незнакомые слова по "похожести" на присутствующие в тренировочном наборе, в том числе по окончаниям и т.п., что даёт ей шанс верно угадать тэг для таких слов, в то время как модель, обучающаяся на словах, не имеет никаких данных о строение слова, и увидев новое слово не понимает, на что оно может быть похоже.